In [4]:
!pip install requests openai


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
!pip install python-dotenv

  Obtaining dependency information for python-dotenv from https://files.pythonhosted.org/packages/6a/3e/b68c118422ec867fa7ab88444e1274aa40681c606d59ac27de5a5588f082/python_dotenv-1.0.1-py3-none-any.whl.metadata



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import os
import requests
from dotenv import load_dotenv

# Load .env file
load_dotenv()

# Access the API key
news_api_key = os.getenv("NEWS_API_KEY")

def fetch_news(topic="technology", language="en"):
    """
    Fetches the latest news articles from NewsAPI based on topic.
    
    Parameters:
        topic (str): Category of news to fetch.
        
    Returns:
        list: A list of dictionaries, each containing details of a news article.
    """
    
    url = "https://newsapi.org/v2/top-headlines"
    params = {
        "apiKey": news_api_key,
        "category": "technology"
    }
    
    # Make the request to NewsAPI
    response = requests.get(url, params=params)
    
    # Check for successful response
    if response.status_code == 200:
        data = response.json()
        # Extract articles from the response
        articles = data.get("articles", [])
        # Format the articles in a simpler structure
        formatted_articles = [
            {
                "title": article["title"],
                "description": article["description"],
                "url": article["url"],
                "publishedAt": article["publishedAt"]
            }
            for article in articles
        ]
        return formatted_articles
    else:
        # Handle errors and display a message if the request fails
        print(f"Error fetching news: {response.status_code}")
        return []

# Test the function to fetch tech news
news_articles = fetch_news()
print(news_articles[:5])  # Print the first 5 articles for testing

[{'title': 'Of course someone has Doom running on Alarmo now - Eurogamer', 'description': "Earlier this week, Tom told us Nintendo Alarmo couldn't yet run Doom, but did confirm it could display a picture of a c…", 'url': 'https://www.eurogamer.net/of-course-someone-has-doom-running-on-alarmo-now', 'publishedAt': '2024-11-02T20:12:54Z'}, {'title': 'Kindle Colorsoft owners complain of a yellow bar on the e-reader’s screen - The Verge', 'description': 'People who’ve ordered Amazon’s color e-reader, the Kindle Colorsoft Signature Edition, say there’s a discolored yellow bar at the bottom of the display.', 'url': 'https://www.theverge.com/2024/11/2/24286289/kindle-colorsoft-yellow-bar-display-e-reader', 'publishedAt': '2024-11-02T20:08:50Z'}, {'title': 'Poll: Are You Happy with Your PS Plus Essential Games for November 2024? - Push Square', 'description': 'Plus or minus?', 'url': 'https://www.pushsquare.com/features/poll-are-you-happy-with-your-ps-plus-essential-games-for-november-2024', 'p

In [8]:
# Load Hugging Face API key from environment variable
huggingface_api_key = os.getenv("HUGGINGFACE_API_KEY")

def summarize_text_with_huggingface(text):
    """
    Summarizes the given text using Hugging Face's Inference API for a summarization model.
    
    Parameters:
        text (str): The text to be summarized.
        
    Returns:
        str: A summarized version of the input text.
    """
    # Define the Hugging Face model endpoint for summarization
    api_url = "https://api-inference.huggingface.co/models/facebook/bart-large-cnn"
    headers = {"Authorization": f"Bearer {huggingface_api_key}"}
    payload = {"inputs": text}
    
    try:
        # Make the API request
        response = requests.post(api_url, headers=headers, json=payload)
        response.raise_for_status()
        
        # Extract the summary from the response
        summary = response.json()[0]["summary_text"]
        return summary
    except requests.exceptions.RequestException as e:
        print(f"Error during summarization: {e}")
        return ""

# Test the function with a sample news article text
sample_text = """
Apple has unveiled its latest iPhone model, featuring a more powerful chip and a revamped camera system. 
The new device, which was announced at the company's annual event, promises better battery life and faster processing speeds. 
Analysts believe these upgrades could help Apple maintain its competitive edge in the smartphone market.
"""
summary = summarize_text_with_huggingface(sample_text)
print("Summary:", summary)

Error during summarization: 400 Client Error: Bad Request for url: https://api-inference.huggingface.co/models/facebook/bart-large-cnn
Summary: 
